# PyTorch `view()` 함수 - 완벽 이해하기

`view()`는 텐서의 **형태를 바꾸는 함수**입니다. 멀티헤드 어텐션에서 중요하게 사용됩니다.

## 🎯 기본 원칙

```python
tensor.view(new_shape)
# 전체 원소 개수는 유지하면서 형태만 변경
```

**핵심**: 원소 개수가 같으면 어떤 형태든 변경 가능!

In [1]:
import torch
import numpy as np

print("PyTorch view() 함수 튜토리얼")
print("=" * 60)

PyTorch view() 함수 튜토리얼


## 예제 1️⃣: 1D → 2D 변환

가장 간단한 1차원 배열을 2차원으로 변환하기

In [2]:
# 원본 (6개 원소)
x = torch.tensor([1, 2, 3, 4, 5, 6])
print(f"원본 텐서: {x}")
print(f"원본 shape: {x.shape}")
print(f"원소 개수: {x.numel()}개")

# 2D로 변경
x_2d = x.view(2, 3)
print(f"\n변경 후 (2, 3):")
print(x_2d)
print(f"새로운 shape: {x_2d.shape}")
print(f"원소 개수: {x_2d.numel()}개 (동일!)")

원본 텐서: tensor([1, 2, 3, 4, 5, 6])
원본 shape: torch.Size([6])
원소 개수: 6개

변경 후 (2, 3):
tensor([[1, 2, 3],
        [4, 5, 6]])
새로운 shape: torch.Size([2, 3])
원소 개수: 6개 (동일!)


**시각적으로:**
```
[1, 2, 3, 4, 5, 6]  (6개 원소)
       ↓
[[1, 2, 3],
 [4, 5, 6]]         (2×3 = 6개 원소)
```

## 예제 2️⃣: 2D → 3D 변환

더 복잡한 형태로 변환하기

In [ ]:
# 12개 원소를 가진 1D 배열
x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
print(f"원본: shape={x.shape}, 원소={x.numel()}개")

# (2, 6) 형태로
x_2d = x.view(2, 6)
print(f"\nview(2, 6):")
print(x_2d)

# (2, 2, 3) 형태로 (12개)
x_3d = x.view(2, 2, 3)
print(f"\nview(2, 2, 3):")
print(x_3d)
print(f"shape: {x_3d.shape}")

# (3, 2, 2) 형태로
x_3d_alt = x.view(3, 2, 2)
print(f"\nview(3, 2, 2):")
print(x_3d_alt)
print(f"shape: {x_3d_alt.shape}")

원본: shape=torch.Size([12]), 원소=12개

view(2, 6):
tensor([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12]])

view(2, 2, 3):
tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]]])
shape: torch.Size([2, 2, 3])

view(3, 2, 2):
tensor([[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]],

        [[ 9, 10],
         [11, 12]]])
shape: torch.Size([3, 2, 2])


**중요한 관찰:**
- 같은 12개 원소도 (2,3), (2,2,3), (3,2,2) 등 다양한 형태로 변환 가능
- 원소 개수: 2×6 = 2×2×3 = 3×2×2 = 12개 (모두 동일)

## 예제 3️⃣: -1 사용 (자동 계산)

`-1`은 "나머지 차원을 자동으로 계산해줘"라는 의미

In [5]:
x = torch.randn(2, 3, 4)
print(f"원본: shape={x.shape}")
print(f"원소 개수: {x.numel()}개\n")

# -1로 자동 계산
x_reshaped = x.view(-1, 6)  # (?, 6)
print(f"view(-1, 6): shape={x_reshaped.shape}")
print(f"계산: 24 = ? × 6 → ? = 4\n")

# 1D로 평탄화
x_flat = x.view(-1)
print(f"view(-1): shape={x_flat.shape}")
print(f"모든 원소가 1D로 펴짐\n")

# 여러 -1 사용 (한 번만 가능)
try:
    x.view(-1, -1)  # 두 개의 -1 ❌
except RuntimeError as e:
    print(f"❌ 두 개의 -1 사용 불가: {e}")

원본: shape=torch.Size([2, 3, 4])
원소 개수: 24개

view(-1, 6): shape=torch.Size([4, 6])
계산: 24 = ? × 6 → ? = 4

view(-1): shape=torch.Size([24])
모든 원소가 1D로 펴짐

❌ 두 개의 -1 사용 불가: only one dimension can be inferred


## 예제 4️⃣: 멀티헤드 분할 (코드의 실제 사용)

멀티헤드 어텐션에서 사용되는 패턴

In [6]:
# 쿼리 벡터 (멀티헤드 어텐션)
queries = torch.tensor([[[0.1, 0.3],
                         [0.2, 0.4],
                         [0.3, 0.5]]], dtype=torch.float32)

print("원본 queries:")
print(f"shape: {queries.shape}  (batch=1, tokens=3, d_out=2)")
print(f"원소 개수: {queries.numel()}개\n")

# 멀티헤드로 분할
# d_out=2를 (num_heads=2, head_dim=1)로 분할
queries_multi = queries.view(1, 3, 2, 1)

print("분할 후 queries_multi:")
print(f"shape: {queries_multi.shape}  (batch=1, tokens=3, heads=2, head_dim=1)")
print(f"원소 개수: {queries_multi.numel()}개 (동일!)\n")

print("토큰 0을 헤드별로 확인:")
print(f"헤드 0: {queries_multi[0, 0, 0, 0].item()}")
print(f"헤드 1: {queries_multi[0, 0, 1, 0].item()}")
print(f"\n원본에서: {queries[0, 0, :]}")
print("→ 첫 값(0.1)은 헤드 0, 두 번째 값(0.3)은 헤드 1으로 분할됨!")

원본 queries:
shape: torch.Size([1, 3, 2])  (batch=1, tokens=3, d_out=2)
원소 개수: 6개

분할 후 queries_multi:
shape: torch.Size([1, 3, 2, 1])  (batch=1, tokens=3, heads=2, head_dim=1)
원소 개수: 6개 (동일!)

토큰 0을 헤드별로 확인:
헤드 0: 0.10000000149011612
헤드 1: 0.30000001192092896

원본에서: tensor([0.1000, 0.3000])
→ 첫 값(0.1)은 헤드 0, 두 번째 값(0.3)은 헤드 1으로 분할됨!


## ⚠️ 주의사항 1️⃣: 원소 개수 확인

원소 개수가 맞지 않으면 에러 발생

In [7]:
x = torch.randn(2, 3)  # 6개 원소
print(f"원본: shape={x.shape}, 원소={x.numel()}개\n")

# 올바른 변환
x_ok = x.view(3, 2)  # 3×2 = 6개 ✓
print(f"✓ view(3, 2): shape={x_ok.shape} (6개 = 6개)")

# 잘못된 변환
try:
    x_error = x.view(2, 4)  # 2×4 = 8개 ❌
    print("이 줄은 실행되지 않음")
except RuntimeError as e:
    print(f"\n❌ view(2, 4) 에러:")
    print(f"   {e}")
    print(f"\n   이유: 6개 원소를 8개 크기로 변환 불가!")

원본: shape=torch.Size([2, 3]), 원소=6개

✓ view(3, 2): shape=torch.Size([3, 2]) (6개 = 6개)

❌ view(2, 4) 에러:
   shape '[2, 4]' is invalid for input of size 6

   이유: 6개 원소를 8개 크기로 변환 불가!


## ⚠️ 주의사항 2️⃣: contiguous() - 메모리 배치

transpose 후에는 메모리 배치가 바뀌어 view()가 실패할 수 있음

In [ ]:
x = torch.randn(3, 4, 5)
print(f"원본: shape={x.shape}")
print(f"is_contiguous: {x.is_contiguous()}\n")

# transpose 후
y = x.transpose(0, 1)  # (4, 3, 5)
print(f"transpose(0,1) 후: shape={y.shape}")
print(f"is_contiguous: {y.is_contiguous()}  ← 이제 불연속!\n")

# 직접 view() 시도 (실패할 수 있음)
try:
    z = y.view(12, 5)  # 시도
    print(f"✓ view(12, 5) 성공: {z.shape}")
except RuntimeError as e:
    print(f"❌ view(12, 5) 실패: {e}\n")
    print("해결책: contiguous() 사용")
    y_contiguous = y.contiguous()
    z = y_contiguous.view(12, 5)
    print(f"✓ contiguous().view(12, 5) 성공: {z.shape}")

## 📋 비교: view() vs reshape() vs flatten()

각 함수의 특징과 차이점

In [ ]:
x = torch.randn(2, 3, 4)
print(f"원본: shape={x.shape}\n")

# 1. view() - 빠름, 연속 메모리 필요
try:
    x_view = x.view(6, 4)
    print(f"✓ view(6, 4): {x_view.shape}")
except RuntimeError:
    print(f"❌ view(6, 4): 실패")

# 2. reshape() - 느림, 자동 contiguous 처리
x_reshape = x.reshape(6, 4)
print(f"✓ reshape(6, 4): {x_reshape.shape}")

# 3. flatten() - 1D로 평탄화만
x_flatten = x.flatten()
print(f"✓ flatten(): {x_flatten.shape}")

print("\n특징 비교:")
print("-" * 50)
print(f"{'함수':<15} {'속도':<10} {'특징'}")
print("-" * 50)
print(f"{'view()':<15} {'빠름':<10} '연속 메모리 필요'")
print(f"{'reshape()':<15} {'느림':<10} '자동 처리, 권장'")
print(f"{'flatten()':<15} {'중간':<10} '1D로만 변환'")

## 🧪 종합 실습

여러 시나리오를 한 번에 테스트

In [ ]:
print("=" * 70)
print("다양한 head_dim 시나리오 비교")
print("=" * 70)

scenarios = [
    ("현재 코드", 2, 2, 1),
    ("변형 1", 4, 2, 2),
    ("변형 2", 8, 4, 2),
    ("GPT-124M", 768, 12, 64),
]

for name, d_out, num_heads, expected_head_dim in scenarios:
    head_dim = d_out // num_heads
    assert head_dim == expected_head_dim, f"head_dim 계산 오류: {head_dim} != {expected_head_dim}"
    
    # 더미 데이터 생성
    batch_size = 1
    num_tokens = 3
    queries = torch.randn(batch_size, num_tokens, d_out)
    
    # 분할
    queries_multi = queries.view(batch_size, num_tokens, num_heads, head_dim)
    
    print(f"\n[{name}]")
    print(f"  d_out={d_out}, num_heads={num_heads}, head_dim={head_dim}")
    print(f"  분할 공식: {d_out} = {num_heads} × {head_dim}")
    print(f"  분할 전: {queries.shape}")
    print(f"  분할 후: {queries_multi.shape}")
    print(f"  원소 개수: {queries.numel()} → {queries_multi.numel()} (동일 ✓)")

## 💡 핵심 정리

### view() 함수의 3가지 원칙

1. **원소 개수 유지**
   - view() 전후로 총 원소 개수는 같아야 함
   - (2, 3, 4) = 24개, view(6, 4) = 24개 ✓

2. **메모리 연속성 확인**
   - transpose() 후에는 contiguous() 필수
   - 그렇지 않으면 RuntimeError 발생 가능

3. **멀티헤드 분할에 사용**
   - d_out을 (num_heads × head_dim)으로 분할
   - 각 헤드가 독립적으로 동작하게 함

### 사용 권장사항

| 상황 | 추천 함수 |
|------|----------|
| 메모리 배치 확실함 | `view()` |
| 메모리 배치 불명확 | `reshape()` |
| 1D로만 변환 | `flatten()` |

In [ ]:
# 최종 확인: 원본과 같은 값인지 검증
print("\n최종 검증: 분할 후 복원")
print("=" * 50)

original = torch.randn(1, 3, 8)
print(f"원본: shape={original.shape}")

# 8 = 4 헤드 × 2 head_dim
reshaped = original.view(1, 3, 4, 2)
print(f"분할: shape={reshaped.shape}")

# 다시 복원
restored = reshaped.view(1, 3, 8)
print(f"복원: shape={restored.shape}")

# 값이 동일한지 확인
is_same = torch.allclose(original, restored)
print(f"\n원본과 복원된 값이 동일? {is_same} ✓")

if is_same:
    print("✅ view()는 단순히 형태만 변경하고 값은 보존합니다!")